In [1]:
import pandas as pd
#from pandas.io.parsers import ParserError
import numpy as np
from helper import get_mapper
import json
import os
import re

In [2]:
from os import listdir, stat
from os.path import isfile, join
BASE_DIR = "."
MIN_SIZE = 512

In [3]:
def extract_blockidf(fullname):
    return fullname.split("Generation_DE ")[1].rsplit('[MW]')[0].strip()

def extract_blockidf2(fullname, plantname):
    return fullname.split("Generation_DE ")[1].rsplit('[MW]')[0].strip().removeprefix(plantname + " ")

def get_smard_name(f):
    return f.rsplit("/")[2].rsplit("_", 4)[0].strip()

In [4]:
get_smard_name("./2015/Abwinden-Asten_201501010000_201512312359_hour_1.csv")

'Abwinden-Asten'

In [5]:
def get_files_from_folder(folder):
    onlyfiles = [folder + "/" + f for f in listdir(folder) if isfile(join(folder, f))]
    onlyfiles.sort()
    files = [f for f in onlyfiles if stat(f).st_size > MIN_SIZE]
    return files

In [6]:
def convert2plantid(df, plantname):
    oldcols = list(df.columns)
    newcols = [extract_blockidf2(x, plantname) for x in list(df.columns)[1:]]
    #print(plantname + ":" + str(newcols))
    try:
        newcols2 = ['produced_at'] + [mapper[plantname][x] for x in newcols]
    except KeyError:
        newcols2 = ['produced_at'] + [mapper[plantname][x.split(" ")[-1]] for x in newcols]
    test = dict(zip(oldcols, newcols2))
    result = df.rename(columns=test)
    #print(oldcols)
    #print(newcols2)
    return result

In [7]:
def get_plant_from_prod_name(prodname):
    tmp = ""
    try:
        tmp = mapper[prodname]
    except KeyError:
        #print("plant " + prodname + " not found!")
        return None, None
    blocklist = tmp['list']
    block = blocklist[0]
    blockid = tmp[block]
    
    try:
        plantidx = bpm.loc[bpm.blockid == blockid, 'plantid'].item()
    except ValueError:
        return blockid, np.nan
    
    return blockid, plantidx

In [8]:
def get_plant_from_prod_name2(prodname):
    tmp = ""
    try:
        tmp = newmapper[prodname]
    except KeyError:
        #print("plant " + prodname + " not found!")
        return None, None
    seelist = tmp['list']
    block = seelist[0]
    blockid = tmp[block]
    
    try:
        plantidx = seem.loc[seem.sseid == blockid, 'plantid'].item()
    except ValueError:
        return blockid, np.nan
    
    return blockid, plantidx

In [9]:
#bpm = pd.read_csv("../basic/block_plant_mapper.csv")
seem = pd.read_csv("../basic/plant_sse_mapper.csv")
#bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))
seem['plantid'] = seem['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [10]:
FOLDERS = [os.path.join(BASE_DIR, o) for o in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR,o))]
FOLDERS.sort()
FOLDERS = FOLDERS[1:-3]
#FOLDERS = FOLDERS[0:1]

In [11]:
#FOLDERS

In [12]:
FILES_L = [get_files_from_folder(f) for f in FOLDERS]
FILES = [item for sublist in FILES_L for item in sublist]

In [13]:
#FILES

In [14]:
#mapper = get_mapper('../production/plantmapper.json')
mapper = get_mapper('newmapper.json')
newmapper = get_mapper('newmapper.json')

In [15]:
#get_plant_from_prod_name("Buschhaus")

In [16]:
get_plant_from_prod_name2("Datteln")

('BNA0189', '06-05-500-0915123')

In [17]:
#mapper

In [18]:
seem

,plantid,sseid
0,01-50-01000004838,SEE957285549260
1,01-50-01000004838,SEE990438768378
2,01-50-01000004838,SEE966920434281
3,01-50-01000004838,SEE999539592561
4,01-50-01000004838,BNA0273
...,...,...
940,06-10-00101000,SEE943764365964
941,01-50-04000082392,SEE953266686481
942,01-50-04000082392,SEE934591835126
943,01-50-04000082392,SEE961021885321


In [19]:
#FILES

In [20]:
#FILES

In [21]:
def get_year(fn):
    return int(fn.split("/")[1])

In [22]:
FILES[0]

'./2015/Abwinden-Asten_201501010000_201512312359_hour_1.csv'

In [23]:
for f in FILES:
    fn = get_smard_name(f)
    #print(fn)

In [24]:
prod_mapper = []
for f in FILES:
    fn = get_smard_name(f)
    #print(fn)
    blockid, plantid = get_plant_from_prod_name2(fn)
    #print(fn or "" + ":" + blockid or "" + "->" + plantid or "")
    year = 2015
    try:
        year = get_year(f)
    except IndexError:
        print(fn)
        pass
    prod_mapper.append([f, blockid, plantid, year])

In [25]:
get_year("./2015/Boxberg_201501010000_201512312345_71.csv")

2015

In [26]:
df = pd.read_csv("./2021/Isar_2_202101010000_202112312359_stunde_81.csv", delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Date", "Start"]], on_bad_lines='skip')

In [27]:
df = df.drop('End', axis=1)

In [28]:
df.fillna(0, inplace=True)
df[df.columns[1:]] = df[df.columns[1:]].astype(int)

In [29]:
df[df.columns[1:]] = df[df.columns[1:]].astype(int)

In [30]:
#df

In [31]:
#df.sort_values(by='Generation_DE KKI 2[MW]')

In [32]:
#prod_mapper 

In [33]:
prd_df = pd.DataFrame(prod_mapper, columns = ['file', 'blockid', 'plantid', 'year'])

In [34]:
prd_df

,file,blockid,plantid,year
0,./2015/Abwinden-Asten_201501010000_20151231235...,None,None,2015
1,./2015/Altenw_rth_201501010000_201512312359_ho...,None,None,2015
2,./2015/Bergkamen_201501010000_201512312359_hou...,SEE930673793902,06-05-900-0205103,2015
3,./2015/Boxberg_201501010000_201512312359_hour_...,SEE968582186379,14-70-46630660001,2015
4,./2015/Braunkohlekraftwerk_Lippendorf_20150101...,SEE966331411864,14-80-00009560000,2015
...,...,...,...,...
1001,./2022/Weisweiler_202201010000_202212312359_st...,BNA1022,06-05-300-0877384,2022
1002,./2022/Wien-Freudenau_202201010000_20221231235...,None,None,2022
1003,./2022/Witznau_202201010000_202212312359_stund...,SEE919035876514,NaN,2022
1004,./2022/Ybbs-Persenbeug_202201010000_2022123123...,None,None,2022


In [35]:
dedup = prd_df.dropna(subset=['blockid', 'plantid'])

In [36]:
#dedup.loc[dedup["year"] == 2022]

In [37]:
#prd_df

In [38]:
#prd_df

In [39]:
for index, row in list(dedup.iterrows()):
    dfname = row[0]
    plantid = str(row[2])
    year = str(int(row[3]))
    smardname = get_smard_name(dfname)
    try:
        df = pd.read_csv(dfname, delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Datum", "Anfang"]], on_bad_lines='skip')
        df = df.drop('Ende', axis=1)
    except ValueError:
        df = pd.read_csv(dfname, delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Date", "Start"]], on_bad_lines='skip')
        df = df.drop('End', axis=1)
    try:
    #print(smardname)
        newdf = convert2plantid(df, smardname)
        newdf.fillna(0, inplace=True)
        newdf[newdf.columns[1:]] = newdf[newdf.columns[1:]].astype(int)
        newdf.to_csv("./by_plantid/" + year + "/" + plantid + '.csv', index=False)
    except (IndexError, KeyError, ValueError):
        print(smardname)
        continue
    
    #try:
    #print(dfname, year)
    #print(type(year))
    
    #except TypeError:
    #    print(year, smardname)
    #print(newdf.dtypes)

Lichterfelde
Lichterfelde
Lichterfelde
Lichterfelde
Lichterfelde


In [40]:
extract_blockidf('Generation_DE Bergkamen A [MW] Originalauflösungen\n')

'Bergkamen A'